# 4. PREPROCESADO

## 4.1. Imports y settings previos

In [ ]:
#Estas dos librerias hay que subirlas al colab nos la proporciono la profesora
!pip install -r requirements.txt
!pip install utils.py
!pip install wordcloud

  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.1
    Uninstalling numpy-1.23.1:
      Successfully uninstalled numpy-1.23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [ ]:
!pip install mxnet-mkl==1.6.0 numpy==1.23.1

  Using cached numpy-1.23.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.1 which is incompatible.


In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
from collections import Counter # Exploración

from stop_words import get_stop_words # Preprocesado
import unicodedata # Preprocesado
import re # Preprocesado
from wordcloud import WordCloud, STOPWORDS


import joblib
from sklearn.model_selection import train_test_split # Modelado
from sklearn.pipeline import Pipeline # Modelado
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # Modelado
from sklearn.feature_selection import chi2 # Reporte
from sklearn.linear_model import LogisticRegression # Reporte
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve # Reporte
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import sklearn.preprocessing as pr
%matplotlib inline

# BeautifulSoup to easily remove HTML tags
from bs4 import BeautifulSoup




 #NLTK library for the remaining steps
import nltk
nltk.download("stopwords")   # download list of stopwords (only once; need not run it again)
from nltk.corpus import stopwords # import stopwords
from nltk import ngrams # Exploración
from nltk.probability import FreqDist # Exploración
from nltk.stem.porter import *
stemmer = PorterStemmer()
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 4.2 Funciones previa de preprocesamiento

El objetivo es crear una sola funcion que realize todo el preprocesamiento, como esta tarea es grande vamos a dividir esa funcion en pequenas funciones que luego uniremos en el siguiente apartado

In [ ]:
def review_to_words(review):
    """Convert a raw review string into a sequence of words."""

    # Eliminamos las etiquetas HTML
    text = BeautifulSoup(review, "html5lib").get_text()
    # Convertimos a minúscula y quitamos todo lo que no sea texto o números
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Dividimos en tokens por espacios
    words = text.split()
    # Eliminamos stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # Aplicamos stemming
    words = [PorterStemmer().stem(w) for w in words]

    return words



In [ ]:
cache_dir = os.path.join("cache", "sentiment_analysis")
os.makedirs(cache_dir, exist_ok=True)

def preprocess_data(data_train, data_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass

    if cache_data is None:
        words_train = list(map(review_to_words, data_train))
        words_test = list(map(review_to_words, data_test))

        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        words_train, words_test = (cache_data['words_train'],
                cache_data['words_test'])

    return words_train, words_test




In [ ]:
def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""

    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass

    if cache_data is None:
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        features_test = vectorizer.transform(words_test).toarray()

        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])

    return features_train, features_test, vocabulary




## 4.3 Funcion de preprocesamiento

In [ ]:
def preprocesamiento(data_train, data_test):
  words_train, words_test = preprocess_data(data_train, data_test)
  features_train, features_test, vocabulary = extract_BoW_features(words_train, words_test)
  features_train = pr.normalize(features_train, axis=1)
  features_test = pr.normalize(features_test, axis=1)
  return features_train, features_test